# Theme detection : From Excel to Wikipidia

_Author: Nikola LOHINSKI_

<div class="alert alert-info" style="margin-top: 1em">
In this notebook, you will learn how to :
<br>
<ul style="list-style: none; padding: 0;">
<li>[&#128279;](#1-Parsing-excel-sheets) **Parse an excel sheet** file in python using the open source library **openpyxl** ;</li>
<li>[&#128279;](#2-Preprocessing-text-data) **Preprocess text** data using using the open source libraries **nltk** and **pyenchant** ;</li>
<li>[&#128279;](#3-Querying-Wikipedia-API) **Querying Wikipedia** API using the open source library **wikitools** in order to retrieve pages, and especially page categories ;</li>
<li>[&#128279;](#4-Detecting-themes-in-short-texts) **Detect themes in short texts** contained in excel cells using the 3 previous points.</li>
</ul>
</div>


<div class="alert alert-warning">
To run this notebook, you will need :
<br>
<ul>
<li>to run on **python 2.7.12** ;</li>
<li>to have **openpyxl** installed. Run '**pip install openpyxl**' in the python environment you are using for this notebook ;</li>
<li>to have **pyenchant** installed. Run '**pip install pyenchant**' in the python environment you are using for this notebook ;</li>
<li>to have **wikitools** installed. Run '**pip install wikitools
**' in the python environment you are using for this notebook.</li>
<li>to have **nltk** installed. Run '**pip install nltk
**' in the python environment you are using for this notebook.</li>
</ul>
</div>

Let's start with the imports :

In [1]:
import json
import csv
import enchant
import nltk
from openpyxl import load_workbook
from functools import reduce
from wikitools import Wiki
from wikitools import APIRequest

We also need to download resource for text data preprocessing :

In [4]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/nikola/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/nikola/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# 1 Parsing excel sheets

The library **openpyxl** is pretty straightforward The file is loaded and explored the same way it is done when using Google Online Sheet or Microsoft's Excel tool. Let's take a look on an example and display the content of all the cells in a column :

In [5]:
# Load excel file
workbook = load_workbook('Fake_EXCEL.xlsx')
# Get active worksheet
worksheet = workbook.active
# Get as specific column
column = worksheet['B']
texts = list()
for i, lign in enumerate(column):
    if i > 0:
        texts.append(lign.value)
print('For example lign 0:\n\n{}'.format(texts[0]))

For example lign 0:

The crew of a colony ship, bound for a remote planet, discover an uncharted paradise with a threat beyond their imagination, and must attempt a harrowing escape. 


That's nice and easy, now let's see how to transform the extracted data to make it neat and clean.

# 2 Preprocessing text data

In here you will get a sense of how to preprocess text data in order to prepare it for further work. The underlying objective is to clear the sentences and words from anything that is not useful and may endanger whatever comes after in the processing pipeline.

### 2.1 Tokenizing, lowercasing, and filter punctuation
First, we need to cast the text a list of lowercased words and remove any left punctuation :

In [6]:
tokenizer = nltk.tokenize.RegexpTokenizer(r'[a-zA-Z]+')
texts_t = list()
for i, text in enumerate(texts):
    texts_t.append(tokenizer.tokenize(text.lower()))
print('For example lign 0 is now:\n\n{}'.format(texts_t[0]))

For example lign 0 is now:

[u'the', u'crew', u'of', u'a', u'colony', u'ship', u'bound', u'for', u'a', u'remote', u'planet', u'discover', u'an', u'uncharted', u'paradise', u'with', u'a', u'threat', u'beyond', u'their', u'imagination', u'and', u'must', u'attempt', u'a', u'harrowing', u'escape']


### 2.2 Removing mispelled words
Then, we shall remove words mispelled therefore not present in a dictionary :

In [7]:
dictionary = enchant.Dict('en_US')
texts_t_d = list()
for i, text in enumerate(texts_t):
    texts_t_d.append(list(filter(lambda w: dictionary.check(w), text)))
print('For example lign 0 is now:\n\n{}'.format(texts_t_d[0]))

For example lign 0 is now:

[u'the', u'crew', u'of', u'a', u'colony', u'ship', u'bound', u'for', u'a', u'remote', u'planet', u'discover', u'an', u'uncharted', u'paradise', u'with', u'a', u'threat', u'beyond', u'their', u'imagination', u'and', u'must', u'attempt', u'a', u'harrowing', u'escape']


<div class="alert alert-info">
**Note** : to improve this part, when using email that can have a lot of words mispelled, it is possible to use auto-correctors and try correct unknown words, but one has to be carefull regarding correcting contact name for example.
</div>

### 2.3 Filtering out _stopwords_
Finally, we remove any _stopword_ in the sentence. A _stopword_ is a word without any semantic content : 'the', 'a', 'not' etc... It carries semantic information but only to link other words among themselves, and is therefore not a standalone entity : 

In [8]:
# Remove stopwords
stopwords = set(nltk.corpus.stopwords.words('english'))
texts_t_d_f = list()
for text in texts_t_d:
    texts_t_d_f.append(list(filter(lambda w: w not in stopwords, text)))
print('For example lign 0:\n\n{}'.format(texts_t_d_f[0]))

For example lign 0:

[u'crew', u'colony', u'ship', u'bound', u'remote', u'planet', u'discover', u'uncharted', u'paradise', u'threat', u'beyond', u'imagination', u'must', u'attempt', u'harrowing', u'escape']


Our text data is now ready for processing. But to be able to query Wikipedia and detect themes, we need to learn how to use its API.

# 3 Querying Wikipedia API

To query Wikipedia's API, several open source libraries are available. Here we use wikitools because it permits very precise queries to be built, and we need to be able to pass specific search parameters. We will see how to search for a wikipdia page and retrieve its categories.

### 3.1 Page search

Let's try for example to look for the word `crew` on Wikipedia :

In [53]:
%%time

site = Wiki("https://en.wikipedia.org/w/api.php")
params = {
    "action": "query",
    "list": "search",
    "srsearch": 'crew'
}

request = APIRequest(site, params)
r = request.query()
result = r['query']['search']

titles = {p['pageid']: p['title'] for p in result}

print(titles.values())
print('-----------------------------------------------------')

[u'Crew neck', u'J.Crew', u'Film crew', u'Motley crew', u'LBC Crew', u'Crew', u'Cutting Crew', u'Crew (disambiguation)', u'Amanda Crew', u'The Crew']
-----------------------------------------------------
CPU times: user 4.82 ms, sys: 3.17 ms, total: 7.99 ms
Wall time: 718 ms


We retrieved information from several pages (the title is only printed but other metadata is also available) but the query took some time ($\sim 0.5 $ seconds on our computer, with the available fast connection). This could be improved by working with an offline version of Wikipedia, or an even better connection to begin with.

<div class="alert alert-danger">
**Important** : it is important to notice that one of the possible listed pages returned by the query is a **disambiguation page**. Those pages are to be handled with caution since they do not have actual useful categories, but do contain semantic links from one them to another.
</div>

### 3.2 Categories 

Since we are able to find a page using a search query, we can now create a query to retrieve its categories :

In [28]:
# Example
example = titles.items()[0]
print('Lets\'s use for the example the following page :')
print('- title : {}\n- page id : {}'.format(example[1], example[0]))

Lets's use for the example the following page :
- title : Crew neck
- page id : 28236391


In [44]:
%%time

example_id = example[0]

def result_to_themes(result, page_id):
    return [x['title'].replace('Category:', '') for x in result['query']['pages'][str(page_id)]['categories']]

params = {
  "action": "query",
  "pageids": example_id,
  "prop": "categories",
  "clshow": "!hidden"
}
request = APIRequest(site, params)

result = request.query()

themes = result_to_themes(result, example_id)
print('Found following categories for page on "{}":'.format(example[1]))
for t in themes:
    print('- {}'.format(t))
print('--------------------------------------------------')

Found following categories for page on "Crew neck":
- Clothing stubs
- Necklines
- Tops (clothing)
--------------------------------------------------
CPU times: user 5.97 ms, sys: 0 ns, total: 5.97 ms
Wall time: 249 ms


Like the previous point, the request takes some time ($\sim 0.25$ ms) and this may be of an issue when analysing a sentence.

We are now able to get an excel file, extract messages from cells, pre process them into list of useful words, and query Wikipedia with those words : we can start detecting themes in short texts.

# 4 Detecting themes in short texts

<div class="alert alert-danger">
**Important** : the following code cells may take some time to execute depending on your computer performance and Internet connection.
</div>

The underlying idea is inspired by [Theme Based Clustering of Tweets](http://www.cse.iitd.ernet.in/~bagchi/paper-theme-based.pdf) by Rudra M. Tripathy,Shashank Sharma, Sachindra Joshi, Sameep Mehta and Amitabha Bagchi :
- for each meaningful word of a text, we search on Wikipedia for related pages
- for each page found, we determine its categories, which correspond to coverted topics in the document
- we count occurences of themes and output the most found themes

In [67]:
%%time
# Example
example_id = 2
words = texts_t_d_f[example_id]
print('Original text:\n{}\n'.format(texts[example_id]))
print('Preprocessed text:\n{}\n'.format(words))
candidate_pages = dict()
site = Wiki('https://en.wikipedia.org/w/api.php')
# Get pages
for w in words:
    params = { 'action': 'query', 'list': 'search', 'srsearch': w }
    request = APIRequest(site, params)
    r = request.query()
    pages = r['query']['search']
    page_ids = [(p['pageid'], p['title']) for p in pages]
    for x in page_ids:
        page_id, title = x
        if candidate_pages.get(title) is None:
            candidate_pages[title] = page_id

# Get categories
site = Wiki("https://en.wikipedia.org/w/api.php")
candidate_themes = dict()
for p in candidate_pages.values():
    params = { 
        'action': 'query',
        'pageids': p,
        'prop': 'categories',
        'clshow': '!hidden'
    }
    request = APIRequest(site, params)
    result = request.query()
    themes = [x['title'].replace('Category:', '') for x in result['query']['pages'][str(p)]['categories']]
    if "Disambiguation pages" not in themes:
        for t in themes:
            if candidate_themes.get(t) is None:
                candidate_themes[t] = 0
            candidate_themes[t] += 1

# Get results
print('Most probable theme occurences:')
for t in sorted(candidate_themes, key=candidate_themes.get, reverse=True):
    if candidate_themes[t] < 3:
        break
    print(t + ' : {} times'.format(candidate_themes[t]))          
print('--------------------------------------------------')

Original text:
A game designer on the run from assassins must play her latest virtual reality creation with a marketing trainee to determine if the game has been damaged.

Preprocessed text:
[u'game', u'designer', u'run', u'assassins', u'must', u'play', u'latest', u'virtual', u'reality', u'creation', u'marketing', u'trainee', u'determine', u'game', u'damaged']

Most probable theme occurences:
English-language films : 9 times
American films : 9 times
Action-adventure games : 5 times
Assassin's Creed : 5 times
Promotion and marketing communications : 4 times
PlayStation 4 games : 4 times
Directorial debut films : 4 times
Reality by type : 3 times
Open world video games : 3 times
Types of marketing : 3 times
Design : 3 times
Living people : 3 times
2014 video games : 3 times
Creation myths : 3 times
Marketing techniques : 3 times
Digital marketing : 3 times
--------------------------------------------------
CPU times: user 716 ms, sys: 186 ms, total: 902 ms
Wall time: 47.5 s


We finally get a list of themes, ordered by occurence during search phase. Some may not be accurate but in general, with 3 to 4 words, it captures the general underlying theme.